In [ ]:
# Prepare import of modules from parent directory.
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from studies import db
from studies.cycles.lib import query_builder
from studies.cycles.lib.thing import Thing
from preparations import things_provider

tp = things_provider.ThingsProvider()
tp.filter_only_primary_signal_datastreams()
primary_signal_datastream_ids = tp.get_datastream_ids()

tp = things_provider.ThingsProvider()
tp.filter_only_cycle_second_datastreams()
cycle_second_datastream_ids = tp.get_datastream_ids()

datastream_ids = primary_signal_datastream_ids + cycle_second_datastream_ids

WINDOW_SIZE_DB = 100000
WINDOW_SIZE_THINGS = 100
LIMIT = 800000

db = db.DBClient(WINDOW_SIZE_DB)
relevant_observations_query_bike = query_builder.get_relevant_observations(datastream_ids, LIMIT)
relevant_observation_rows_bike_generator = db.execute_query(relevant_observations_query_bike)

tp = things_provider.ThingsProvider()
tp.filter_only_bike_things()
things_and_layer_names_by_datastream_id = tp.get_map_from_datastream_ids_to_thing_names_and_layer_names()

VALIDATION = True
RETRIEVE_ALL_CLEANUP_STATS = True

things = {}

for row in relevant_observation_rows_bike_generator:
    thing_name, layer_name = things_and_layer_names_by_datastream_id[row[2]]
    if thing_name is None:
        raise Exception('Thing name is None for datastream id: ' + str(row[2]))
    if layer_name is None:
        raise Exception('Layer name is None for datastream id: ' + str(row[2]))
    
    if thing_name not in things:
        things[thing_name] = Thing(thing_name, WINDOW_SIZE_THINGS, VALIDATION, RETRIEVE_ALL_CLEANUP_STATS)
        
    things[thing_name].add_observation(layer_name, row[0], row[1])

db.close()
